In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ecommerce-data/data.csv


In [2]:
# Load the UK e-commerce dataset
df = pd.read_csv("/kaggle/input/ecommerce-data/data.csv", encoding='ISO-8859-1')

# Preview the first 5 rows
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [3]:
#explore data 
df.isnull().sum()
#look for missin value 

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [4]:
df_clean = df.dropna(subset=["CustomerID"])
#drop missing value

In [5]:
df_clean["InvoiceDate"] = pd.to_datetime(df_clean["InvoiceDate"])
#Convert InvoiceDate to datetime

/tmp/ipykernel_36/3344600835.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["InvoiceDate"] = pd.to_datetime(df_clean["InvoiceDate"])


In [6]:
df_clean.loc[:, "InvoiceDate"] = pd.to_datetime(df_clean["InvoiceDate"])


In [7]:
df_clean = df_clean.drop_duplicates(subset=["InvoiceNo", "StockCode", "CustomerID"])


In [8]:
df_clean["Revenue"] = df_clean["Quantity"] * df_clean["UnitPrice"]


In [9]:
top_products = df_clean.groupby("Description")["Revenue"].sum().sort_values(ascending=False).head(10)
top_products
#top 10 products by revenue 

Description
REGENCY CAKESTAND 3 TIER              132248.95
WHITE HANGING HEART T-LIGHT HOLDER     93422.65
JUMBO BAG RED RETROSPOT                82984.86
PARTY BUNTING                          67458.18
POSTAGE                                66793.88
ASSORTED COLOUR BIRD ORNAMENT          56110.52
RABBIT NIGHT LIGHT                     50861.88
CHILLI LIGHTS                          45885.71
PAPER CHAIN KIT 50'S CHRISTMAS         41108.13
PICNIC BASKET WICKER 60 PIECES         39619.50
Name: Revenue, dtype: float64

In [19]:
top_revenue = df_clean.groupby("Country")["Revenue"].sum().sort_values(ascending=False).head(10)
top_revenue
#top 10 revebue based on countries 

Country
United Kingdom    6716562.413
Netherlands        284661.540
EIRE               249985.280
Germany            221004.020
France             196516.480
Australia          136976.570
Switzerland         55739.400
Spain               54649.670
Belgium             40910.960
Sweden              36585.410
Name: Revenue, dtype: float64

In [20]:
total_revenue = df_clean.groupby("InvoiceHour")["Revenue"].sum().sort_values(ascending=False).head(5)
total_revenue
#total revenue for each hour of the day. Which hour sees the highest sales volume?

InvoiceHour
12    1326142.240
10    1146165.951
13    1110692.710
11    1049625.260
14     940606.461
Name: Revenue, dtype: float64

In [16]:
df_clean["InvoiceHour"] = df_clean["InvoiceDate"].dt.hour


In [21]:
top_customers = df_clean.groupby("CustomerID")["Revenue"].sum().sort_values(ascending=False).head(10)
top_customers
#What are the top 10 customers  by total revenue?

CustomerID
14646.0    279489.02
18102.0    256438.49
17450.0    187599.49
14911.0    132458.73
12415.0    123725.45
14156.0    113198.09
17511.0     88125.38
16684.0     65892.08
13694.0     62690.54
15311.0     59248.17
Name: Revenue, dtype: float64

In [22]:
cancel_orders = df_clean[df_clean["InvoiceNo"].str.startswith("C")].groupby("Country")["InvoiceNo"].count().sort_values(ascending=False).head(10)
cancel_orders
#Which countries had the most canceled orders?

Country
United Kingdom    7443
Germany            451
EIRE               247
France             147
USA                112
Australia           74
Spain               48
Italy               45
Belgium             38
Japan               37
Name: InvoiceNo, dtype: int64

In [26]:
most_products = df_clean.groupby("Description")["Quantity"].sum().sort_values(ascending=False).head(5)
most_products
#What are the top 5 most frequently purchased products?

Description
WORLD WAR 2 GLIDERS ASSTD DESIGNS     53119
JUMBO BAG RED RETROSPOT               44929
ASSORTED COLOUR BIRD ORNAMENT         35084
WHITE HANGING HEART T-LIGHT HOLDER    34011
PACK OF 72 RETROSPOT CAKE CASES       33332
Name: Quantity, dtype: int64

In [27]:
df_clean.to_csv("ecommerce_clean.csv", index=False)
